In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import random
from operator import add

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [7]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param W {J} >=0;   # number of worker of doing the job
param K {I} >=0;   # capacity of machine 
param R {I} >=0;   # quality factor of machine 
param S {I} >=0;   # speed of machine 

var X {I,J} >=0 binary;   # Decision var 1
var Y {J}   >=0 binary;   # Decision var 2

maximize Benefit: sum {i in I, j in J} R[i] * B[j] * X[i,j];


s.t. CoWork   {        j in J}: Y[j] * W[j] = sum {i in I} X[i,j];

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= S[i] * K[i];
s.t. Arrange1 {i in I, j in J}:                     X[i,j] <= 1;
s.t. Arrange2 {i in I, j in J}:                     X[i,j] >= 0;
s.t. Arrange3 {        j in J}:                     Y[  j] <= 1;
s.t. Arrange4 {        j in J}:                     Y[  j] >= 0;

''')

## Step 5: Define the initial data

In [8]:
machine_job  = [(5,20)]
cowork       = ['O','F','M']     # 
capacity     = ['N','L','T']     # 
relation_bc  = ['L','X','A','R'] # 

In [9]:
# batch 要做成:
# [ num_machine, num_job, cost, benefit, capacity ]

it_n = 100

def batch (num_machine, num_job, relation_bc, capacity, cowork):
    
    m = num_machine
    n = num_job
    
    list_r1 = []    # 內含 100組 m個 machine的 quality factor   組合
    list_s1 = []    # 內含 100組 m個 machine的 speed            組合
    
    list_c1 = []    # 內含 100組 n個 job的 cost                 組合
    list_w1 = []    # 內含 100組 n個 job的 worker               組合
    list_b1 = []    # 內含 100組 n個 job的 benefit              組合
    
    list_k1 = []    # 內含 100組 由 n個 job 加總算出的 k
    
    for h in range(it_n):
        
        list_r2 = []   # 內含 m個 machine的 quality factor
        list_s2 = []   # 內含 m個 machine的 speed        
    
        list_c2 = []   # 內含 n個 job的 cost
        list_w2 = []   # 內含 n個 job的 worker
        list_b2 = []   # 內含 n個 job的 benefit 
                    
        for i in range(m):
            
            r = (random.random())*2 + 1   # 1-3之間亂數  
            s = (random.random())*2 + 1   # 1-3之間亂數
            list_r2.append(r)
            list_s2.append(s)
            
        for j in range(n):
                
            c = int(random.randint(1,50))
            list_c2.append(c)

            if   relation_bc == 'L':
                b = c

            elif relation_bc == 'X':
                b = c**2

            elif relation_bc == 'A':
                b = c**(1/2)

            elif relation_bc == 'R':
                b = random.randint(0,50) 
            
            list_b2.append(b)
            
            
            if   cowork == 'M': # most
                q = m
                
            elif cowork == 'F': # some
                q = int(m*1/2)
                
            elif cowork == 'O': # few
                q = 1
                
            w = random.randint(1, q)
            list_w2.append(w)
            
        if   capacity =='N':
            k = 100000 # no capacity

        elif capacity =='L':
            k = int( sum(list_c2) / m )

        elif capacity =='T':
            k = int( sum(list_c2) * (0.75) / m )

        list_r1.append(list_r2)
        list_s1.append(list_s2)
        list_c1.append(list_c2)
        list_w1.append(list_w2)
        list_b1.append(list_b2)
        list_k1.append(k)
    
    return  m, n, list_c1, list_w1, list_b1, list_k1, list_r1, list_s1

In [10]:
# Test OK! 不要在正式 run 100組時  print，會很慢 >"<
'''
instance = batch( 5, 20, 'A', 'T', 'M' )
print(instance[2])
print(instance[3])
print(instance[4])

print(instance[6])
print(instance[7])
'''

[[13, 26, 6, 18, 33, 25, 50, 35, 35, 46, 29, 2, 27, 25, 37, 11, 20, 9, 47, 1], [17, 19, 14, 35, 50, 31, 23, 38, 27, 16, 35, 7, 1, 49, 50, 8, 45, 31, 34, 46], [4, 9, 50, 37, 37, 26, 3, 37, 19, 32, 30, 19, 18, 42, 17, 26, 18, 41, 47, 41]]
[[3, 1, 1, 5, 5, 2, 4, 1, 3, 5, 5, 4, 2, 4, 1, 3, 5, 2, 3, 5], [5, 2, 2, 5, 5, 1, 1, 3, 4, 4, 4, 2, 5, 5, 3, 5, 3, 1, 5, 5], [5, 2, 3, 4, 4, 5, 5, 2, 1, 3, 2, 4, 5, 2, 2, 3, 4, 2, 5, 4]]
[[3.605551275463989, 5.0990195135927845, 2.449489742783178, 4.242640687119285, 5.744562646538029, 5.0, 7.0710678118654755, 5.916079783099616, 5.916079783099616, 6.782329983125268, 5.385164807134504, 1.4142135623730951, 5.196152422706632, 5.0, 6.082762530298219, 3.3166247903554, 4.47213595499958, 3.0, 6.855654600401044, 1.0], [4.123105625617661, 4.358898943540674, 3.7416573867739413, 5.916079783099616, 7.0710678118654755, 5.5677643628300215, 4.795831523312719, 6.164414002968976, 5.196152422706632, 4.0, 5.916079783099616, 2.6457513110645907, 1.0, 7.0, 7.0710678118654755, 

## Step 6: Model Build-up & Solution

In [10]:
# 迭代 instance

TotalBenefit_ijk_sn = []
instance_list = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]

    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for l in cowork:
        for k in capacity:    
            for j in relation_bc:
                                
                instance = batch( i[0], i[1], j, k, l )
                instance_list.append(instance)

                TotalBenefit_1 = []
                TotalBenefit_2 = []

                for p in range(it_n):    

                    if k == 'N':
                        ca_adj = instance[7][p]
                    else:
                        ca_adj = [1] * num_machine                
                    
                    # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均

                    # amplpy DataFrame
                    ampl.setData(DataFrame(
                    index=[('J', job)], 
                    columns=[
                        ('B', instance[4][p]), 
                        ('W', instance[3][p]), 
                        ('C', instance[2][p])
                    ]
                    ))

                    # Pandas DataFrame
                    df = pd.DataFrame({
                        'K': [instance[5][p]]*num_machine,
                        'R':  instance[6][p],
                        'S':  list(map(lambda a, b: a / b, instance[7][p], ca_adj))
                    }, 
                        index = machine
                    )
                    ampl.setData(DataFrame.fromPandas(df))

                    
                    # Solve the problem
                    ampl.solve()

                    '''
                    # Create a DataFrame with Decision variables
                    Var = ampl.getVariable('X').getValues().toPandas()
                    print(Var)
                    '''

                    # Display the objective value
                    TotalBnft_1 = ampl.getObjective('Benefit')
                    TotalBnft_1 = TotalBnft_1.value()
                    TotalBenefit_1 = np.append(TotalBenefit_1, TotalBnft_1) # numpy.ndarray


                    # ====== Model 2 ======= : 一組一組用 CHBF_W 算，每種senario要算 100組 再平均

                    # 將 job 按照 cost (workload) 排序

                    list_c = instance[2][p]
                    list_w = instance[3][p]
                    list_b = instance[4][p]

                    list_cwb = list(zip(list_c, list_w, list_b)) # [3,5] [4,2] [8,1] -> [(3,4,8),(5,2,1)]
                    s_cwb    = sorted(list_cwb, reverse = True)  # [(3,4,8),(5,2,1)] -> [(5,2,1),(3,4,8)]
                    o_cwb    = list(zip(*s_cwb))                 # [(5,2,1),(3,4,8)] -> [(5,3),(2,4),(1,8)]
                    
                    list_c  = list(o_cwb[0])                    # [5,3]
                    list_w  = list(o_cwb[1])                    # [2,4]
                    list_b  = list(o_cwb[2])                    # [1,8]
                    
                    # 將 machine 按照 SiRi 排序
                    
                    list_r = instance[6][p]
                    list_s = list(map(lambda a, b: a / b, instance[7][p], ca_adj))
                    
                    list_rs = [a*b for a,b in zip(list_r,list_s)]
                    list_rs = list(zip(list_rs, list(range(1,6))))
                    list_rs = sorted(list_rs, reverse = True)

                    list_r_p = []
                    list_s_p = []
                    mach     = [] 
                    ca = instance[5][p]
                    
                    #print(ca)

                    for r in range(num_machine):

                        p = list_rs[r][1]-1
                        list_r_p.append(list_r[p])       # 製作 Ri list
                        list_s_p.append(list_s[p])       # 製作 Si list
                        
                        #print(list_r_p)
                        #print(list_s_p)
                                            
                        mach.append([])                  # machine set中安排一台台 machine 被 append 進來
                        mach[r].append(0)                # 初始的 benefit和 是 0
                        mach[r].append(ca * list_s_p[r]) # 初始的 capacity  是 K*Si
                                              
                    
                    mach_1 = mach
  
                    TotalBnft_2 = 0 
                    
                    for s in range(num_job):
                                                
                        Cs = list_c[s]
                        Ws = list_w[s]
                        Bs = list_b[s]
                        
                        w = 0
                        
                        #print(mach)
                        
                        mach_1 = sorted(mach_1, key=lambda x: x[1], reverse=True)  # 仍有 Ws台以上 machine 有餘裕

                        #print(mach_1)

                        #print(Cs)
                        #print(Ws)
                        #print(mach_1[Ws-1][1])
                        
                        if mach_1[Ws-1][1] >= Cs:              

                            for r in range(num_machine):

                                if (Cs <= mach[r][1] and w < Ws):               # 按 SiRi順序 assign，直到 Ws台 machine都完成

                                    mach[r][1] -= Cs
                                    mach[r][0] += Bs

                                    mach_1 = mach

                                    w += 1

                    for r in range(num_machine):                  
                        TotalBnft_2 += mach[r][0] * list_r_p[r] # 將所有 machine 的 benefit*quality factor 和加起來成為一組
                        
                        #print(mach[r][0])
                        #print(list_r_p[r])
                        #print(TotalBnft_2)

                    TotalBenefit_2 = np.append(TotalBenefit_2, TotalBnft_2) # 將 100組全 append在一起 
                    #print(TotalBenefit_2)


                # ====== Model 1 Objective value: ====== #

                list_M1            = list(TotalBenefit_1)

                # ====== Model 2 Objective value: ====== #

                list_M2            = list(TotalBenefit_2)
                
                #print(list_M1)
                #print(list_M2)

                list_ratio = list(map(lambda a, b: a / b, list_M2, list_M1))
                TotalBenefit_100_1 = sum(list_ratio) / len(list_ratio)
                TotalBenefit_ijk_sn.append(TotalBenefit_100_1)     # i*j*k*l 種 senario 迭代

                print('=========================' + str(i) + str(j) + str(k) + str(l) + '=========================') # 分隔           
            
print(TotalBenefit_ijk_sn)


CPLEX 12.8.0.0: optimal integer solution; objective 5762.704291
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 3443.893639
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4699.710786
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4451.427258
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 3479.85464
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4273.22618
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4898.164719
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4741.561259
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4037.148829
0 MIP simplex iterations
0

CPLEX 12.8.0.0: optimal integer solution; objective 1472.368236
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 2308.726791
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4024.88558
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4345.982818
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4976.143533
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 3880.104206
0 MIP simplex iterations
0 branch-and-bound nodes
=========================(5, 20)LNO=========================
CPLEX 12.8.0.0: optimal integer solution; objective 185458.0125
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 111725.1025
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal inte

CPLEX 12.8.0.0: optimal integer solution; objective 802.930071
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 649.4141603
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 868.8571052
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 942.7195364
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 998.1427348
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 940.3629455
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 557.5582645
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 684.6345275
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 527.7424629
0 MIP simplex iterations


CPLEX 12.8.0.0: optimal integer solution; objective 3823.470924
30 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 4.54747e-13, relmipgap = 1.18936e-16
CPLEX 12.8.0.0: optimal integer solution; objective 3460.78034
26 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4723.667167
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 3713.506888
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 4171.051769
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 3913.827559
31 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 4760.362535
55 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 0.130953, relmipgap = 2.75091e-05
CPLEX 12.8.0.0: optimal integer solution; objective 3202.202407
0 MIP simplex

CPLEX 12.8.0.0: optimal integer solution; objective 177727.679
32 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 147614.3905
137 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 52412.8514
46 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 210206.069
42 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 142793.5108
55 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 118921.0693
40 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 144591.6004
41 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 171685.4128
39 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 128

...

CPLEX 12.8.0.0: optimal integer solution; objective 163498.1583
10 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 211477.2599
27 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 221849.6714
22 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 210527.022
30 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 175979.1302
143 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 2.91038e-11, relmipgap = 1.65382e-16
CPLEX 12.8.0.0: optimal integer solution; objective 242848.7169
13 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 2.91038e-11, relmipgap = 1.19843e-16
CPLEX 12.8.0.0: optimal integer solution; objective 209604.2724
39 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 132914.7196
46 MIP simplex iterations
0 branc

CPLEX 12.8.0.0: optimal integer solution; objective 821.0704366
35 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 1.13687e-13, relmipgap = 1.38462e-16
CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 1332.546247
122 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 0.11964, relmipgap = 8.97827e-05
CPLEX 12.8.0.0: optimal integer solution; objective 979.9275653
32 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 647.9342452
2469 MIP simplex iterations
375 branch-and-bound nodes
absmipgap = 0.062149, relmipgap = 9.59187e-05
CPLEX 12.8.0.0: optimal integer solution; objective 956.1170277
54 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 1.13687e-13, relmipgap = 1.18905e-16
CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 477.0598829
119 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 0.0358814, relmipgap =

CPLEX 12.8.0.0: optimal integer solution; objective 10589.60777
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 9283.677465
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 10160.06212
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 9795.060876
1 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 3.63798e-12, relmipgap = 3.7141e-16
CPLEX 12.8.0.0: optimal integer solution; objective 9871.187463
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 5567.355024
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 10733.10386
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 16022.78088
1 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 3.63798e-12, relmipgap = 2.2

CPLEX 12.8.0.0: optimal integer solution; objective 12041.9806
1 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 1.81899e-12, relmipgap = 1.51054e-16
CPLEX 12.8.0.0: optimal integer solution; objective 10543.33901
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 8989.68096
1 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 1.81899e-12, relmipgap = 2.02342e-16
CPLEX 12.8.0.0: optimal integer solution; objective 7321.799294
1 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 9.09495e-13, relmipgap = 1.24217e-16
CPLEX 12.8.0.0: optimal integer solution; objective 11575.24604
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 7802.934584
1 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 9556.567192
1 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 5.45697e-12, relmipgap = 5.71018e-16
CPLEX 12

CPLEX 12.8.0.0: optimal integer solution; objective 257268.3598
40 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 265520.8183
43 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 183522.2596
28 MIP simplex iterations
0 branch-and-bound nodes
absmipgap = 2.91038e-11, relmipgap = 1.58585e-16
CPLEX 12.8.0.0: optimal integer solution; objective 189914.411
1142 MIP simplex iterations
132 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 286621.0679
24 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 405884.1949
35 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 266937.4799
10 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 290556.2925
35 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal inte

In [11]:
import pickle
with open("instance_list_HSQFCC_1_RS1to3.txt", "wb") as fp:
    pickle.dump(instance_list, fp)

with open("instance_list_HSQFCC_1_RS1to3.txt", "rb") as fp:
    instance_saved = pickle.load(fp)

In [12]:
print(instance_saved[0][7])

[[8.655489286874625, 6.7483133616476705, 1.9345632952296574, 2.432395652451794, 9.961301323545523], [3.8155264261539967, 6.102065149530917, 7.105133292651431, 1.4931296301760917, 5.594967131100636], [2.6449937956353216, 1.6372308204632384, 3.165725798139603, 9.411897491358921, 3.401753458929316], [7.985112847614733, 6.090406324687736, 9.068084414895953, 2.0961100130350645, 4.431162318321636], [6.780811787180022, 6.710741072653899, 9.872278413742691, 2.6250136916418914, 4.502200660402696], [7.524691647185349, 6.322616425313797, 7.771805239444796, 8.699443943585585, 9.326479454106648], [6.490279549347545, 4.222722143107771, 9.38130367275821, 9.157902966030035, 1.7963613378265197], [6.791657199737765, 4.585852404661782, 4.576290947745901, 9.76890271276363, 3.0292401928474586], [9.603775986702097, 7.89076423951132, 5.349995299271524, 5.3513251262641415, 1.5776603552794173], [2.1859861581403544, 1.8358859089535078, 3.6995917243995047, 7.178729350595962, 6.2702442574564], [2.1384859547335346

In [13]:
tag = []
for l in cowork:
    for k in capacity:    
        for j in relation_bc:
            t = [l,k,j]
            tag.append(t)            

z = list(zip(tag, TotalBenefit_ijk_sn))
z

[(['O', 'N', 'L'], 1.0),
 (['O', 'N', 'X'], 1.0),
 (['O', 'N', 'A'], 1.0),
 (['O', 'N', 'R'], 1.0),
 (['O', 'L', 'L'], 0.9263960965286584),
 (['O', 'L', 'X'], 0.90438165529589543),
 (['O', 'L', 'A'], 0.91199474695849181),
 (['O', 'L', 'R'], 0.87230454911578403),
 (['O', 'T', 'L'], 0.92559565406377753),
 (['O', 'T', 'X'], 0.93139045071027426),
 (['O', 'T', 'A'], 0.90245415129930084),
 (['O', 'T', 'R'], 0.87614489291542308),
 (['F', 'N', 'L'], 1.0),
 (['F', 'N', 'X'], 1.0),
 (['F', 'N', 'A'], 1.0),
 (['F', 'N', 'R'], 1.0),
 (['F', 'L', 'L'], 0.93523215346667798),
 (['F', 'L', 'X'], 0.92430521325534543),
 (['F', 'L', 'A'], 0.92869463841007027),
 (['F', 'L', 'R'], 0.91615657004249795),
 (['F', 'T', 'L'], 0.94107191786096689),
 (['F', 'T', 'X'], 0.92797181560458952),
 (['F', 'T', 'A'], 0.90717194458264561),
 (['F', 'T', 'R'], 0.87506486113143933),
 (['M', 'N', 'L'], 1.0),
 (['M', 'N', 'X'], 1.0000012091812014),
 (['M', 'N', 'A'], 1.0),
 (['M', 'N', 'R'], 1.0),
 (['M', 'L', 'L'], 0.956253164

In [ ]:
# 以最小benefit排序

                    for s in range(num_job):

                        mach = sorted(mach)           # 按照 benefit和 由小而大排序

                        Cs = list_c[s]
                        Ws = list_w[s]
                        Bs = list_b[s]

                        if Cs <= mach[Ws-1][1]:       # 至少還有 Ws台 machine 有 capacity 餘裕

                            for t in range(Ws):

                                mach[t][1] -= Cs
                                mach[t][0] += Bs